In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv


In [2]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Reproductibilité
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)

# Style des graphiques
sns.set(style='whitegrid')

2025-10-11 08:38:16.093425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760171896.371959      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760171896.457767      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
# Chargement du dataset
data = pd.read_csv('/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv')
print(f"Taille du dataset : {data.shape}")

# Aperçu des données
data.head()

Taille du dataset : (1526724, 2)


,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


In [5]:
print("\nInfo :")
print(data.info())
print("\nDistribution originale des labels :")
print(data['label'].value_counts())


Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526724 entries, 0 to 1526723
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   label   1526724 non-null  int64 
 1   text    1526724 non-null  object
dtypes: int64(1), object(1)
memory usage: 23.3+ MB
None

Distribution originale des labels :
label
0    771604
1    755120
Name: count, dtype: int64


In [6]:
TARGET_PER_CLASS = 600000

def safe_sample_per_class(df, label_col='label', n=TARGET_PER_CLASS, random_state=SEED):
    parts = []
    for lbl, group in df.groupby(label_col):
        sample_n = min(len(group), n)
        parts.append(group.sample(n=sample_n, random_state=random_state))
    return pd.concat(parts).reset_index(drop=True)

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#', ' ', text)
    text = re.sub(r'[^a-z0-9àâäéèêëïîôöùûüçœæ\-\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Nettoyage
data['text_clean'] = data['text'].astype(str).map(clean_text)

# Échantillonnage équilibré
data_balanced = safe_sample_per_class(data, label_col='label', n=60000)
print("Distribution équilibrée :")
print(data_balanced['label'].value_counts())

data_balanced[['text', 'text_clean', 'label']].head()

Distribution équilibrée :
label
0    60000
1    60000
Name: count, dtype: int64


,text,text_clean,label
0,"Non, je ne la regarde pas, idk elle.",non je ne la regarde pas idk elle,0
1,"- Très, très vrai.",- très très vrai,0
2,Allongé au lit et essayant de dormir ne se pas...,allongé au lit et essayant de dormir ne se pas...,0
3,"S'endormir, rêver à propos de Tom et un amour ...",s endormir rêver à propos de tom et un amour q...,0
4,"Avoir eu des problèmes majeurs avec la santé, ...",avoir eu des problèmes majeurs avec la santé l...,0


In [7]:
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "microsoft/MiniLM-L12-H384-uncased"

print("Chargement du tokenizer et du modèle MiniLM...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

print(f"✅ Modèle chargé : {MODEL_NAME}")
print(f"- Couches: {model.config.num_hidden_layers}")
print(f"- Dimensions cachées: {model.config.hidden_size}")
print(f"- Têtes d'attention: {model.config.num_attention_heads}")

Chargement du tokenizer et du modèle MiniLM...


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

✅ Modèle chargé : microsoft/MiniLM-L12-H384-uncased
- Couches: 12
- Dimensions cachées: 384
- Têtes d'attention: 12


In [8]:
import transformers
from transformers import AutoTokenizer, AutoModel

# Solution 1: Utiliser un modèle MiniLM plus récent et stable
MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
print("Chargement du tokenizer et du modèle MiniLM...")

# Chargement avec paramètres pour éviter les erreurs
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Vérification des spécifications du modèle
print(f"\nArchitecture du modèle MiniLM:")
print(f"- Couches: {model.config.num_hidden_layers}")
print(f"- Dimensions cachées: {model.config.hidden_size}")
print(f"- Têtes d'attention: {model.config.num_attention_heads}")

Chargement du tokenizer et du modèle MiniLM...


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]


Architecture du modèle MiniLM:
- Couches: 12
- Dimensions cachées: 384
- Têtes d'attention: 12


In [9]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_minilm_embeddings(texts, batch_size=32):
    model.eval()
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encoded_input = tokenizer(
            batch_texts.tolist(),
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        with torch.no_grad():
            model_output = model(**encoded_input)
        embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        all_embeddings.extend(embeddings.numpy())
    return np.array(all_embeddings)

In [10]:
test_texts = pd.Series([
    "J'adore ce produit !",
    "Je déteste ce service.",
    "C'était correct, sans plus."
])

emb_test = get_minilm_embeddings(test_texts)
print("Shape des embeddings :", emb_test.shape)
print(emb_test[0][:10])  # premières valeurs pour vérifier

Shape des embeddings : (3, 384)
[-0.22523013  0.5134914   0.187118   -0.3391628  -0.11511293 -0.0803717
  0.26003262  0.203393   -0.22086304 -0.02835667]


In [11]:
SAMPLE_SIZE = min(60000, len(data_balanced))
data_sampled = data_balanced.sample(n=SAMPLE_SIZE, random_state=SEED)

texts = data_sampled['text_clean']
labels = data_sampled['label']

print(f"Taille échantillon : {len(texts)}")

# Extraction embeddings
minilm_embeddings = get_minilm_embeddings(texts)
np.save('/kaggle/working/minilm_embeddings.npy', minilm_embeddings)

print(f"Shape des embeddings : {minilm_embeddings.shape}")

Taille échantillon : 60000
Shape des embeddings : (60000, 384)
